In [25]:
import pandas as pd
merged_ipl_df = pd.read_csv('data/2018_and_2019_ipl_ball_by_ball.csv')

C:\Users\manna\AppData\Local\Temp\ipykernel_14224\94611248.py:2: DtypeWarning:

Columns (20,30) have mixed types. Specify dtype option on import or set low_memory=False.



In [26]:
print(merged_ipl_df['ID'].sum())

new_df = merged_ipl_df[['ID','Team1','Team2','WinningTeam']].copy()
print(new_df['ID'].sum())

33093342724
33093342724


In [27]:
grouped_df = new_df.groupby(new_df.columns.tolist(), sort=False)

In [28]:
wins = {}

# Iterate over dataframe
for group,data in grouped_df:
    team1 = group[1]
    team2 = group[2]
    winner = group[3]

    if (team1,team2) in wins:
        (a,b,sum) = wins[(team1,team2)]
        if(winner == team1):
            wins[(team1,team2)] = (a+1,b,sum+1)
        else:
            wins[(team1,team2)] = (a,b+1,sum+1)
    elif (team2,team1) in wins:
        (a,b,sum) = wins[(team2,team1)]
        if(winner == team1):
            wins[(team2,team1)] = (a,b+1,sum+1)
        else:
            wins[(team2,team1)] = (a+1,b,sum+1)
    else:
        wins[(team1,team2)] = (0,0,0)
        (a,b,sum) = wins[(team1,team2)]
        if(winner == team1):
            wins[(team1,team2)] = (a+1,b,sum+1)
        else:
            wins[(team1,team2)] = (a,b+1,sum+1)

print(wins)

{('Mumbai Indians', 'Chennai Super Kings'): (5, 1, 6), ('Delhi Capitals', 'Chennai Super Kings'): (1, 4, 5), ('Sunrisers Hyderabad', 'Delhi Capitals'): (3, 2, 5), ('Kolkata Knight Riders', 'Mumbai Indians'): (1, 3, 4), ('Chennai Super Kings', 'Kings XI Punjab'): (2, 2, 4), ('Sunrisers Hyderabad', 'Royal Challengers Bangalore'): (2, 2, 4), ('Rajasthan Royals', 'Delhi Capitals'): (1, 3, 4), ('Kings XI Punjab', 'Kolkata Knight Riders'): (1, 3, 4), ('Mumbai Indians', 'Sunrisers Hyderabad'): (2, 2, 4), ('Sunrisers Hyderabad', 'Kings XI Punjab'): (2, 2, 4), ('Delhi Capitals', 'Royal Challengers Bangalore'): (2, 2, 4), ('Sunrisers Hyderabad', 'Rajasthan Royals'): (3, 1, 4), ('Kolkata Knight Riders', 'Rajasthan Royals'): (4, 1, 5), ('Royal Challengers Bangalore', 'Kings XI Punjab'): (4, 0, 4), ('Sunrisers Hyderabad', 'Chennai Super Kings'): (1, 5, 6), ('Royal Challengers Bangalore', 'Chennai Super Kings'): (1, 3, 4), ('Kolkata Knight Riders', 'Sunrisers Hyderabad'): (2, 3, 5), ('Kings XI Punja

In [29]:
wins_separated = {}
for (i,j) in wins.items():
    team1 = i[0]
    team2 = i[1]
    wins_separated[(team1,team2)] = j[0]
    wins_separated[(team2,team1)] = j[1]  # Changed here from 2 to 1 

count = 0
for i in wins_separated:
    count = count+1

print(count)

56


In [30]:
import plotly.graph_objs as go

unique_teams_list = list(set(merged_ipl_df['Team1']).union(set(merged_ipl_df['Team2'])))
# print(len(unique_teams_list)) 

# Create nodes for each team

In [31]:
# Create matrix of head-to-head win counts
matrix = []
for team1 in unique_teams_list:
    row = []
    for team2 in unique_teams_list:
        if team1 == team2:
            row.append(0)
        else:
            key1 = (team1, team2)
            key2 = (team2, team1)
            if key1 in wins_separated:
                row.append(wins_separated[key1])
            elif key2 in wins_separated:
                row.append(wins_separated[key2])
            else:
                row.append(0)
    matrix.append(row)

In [32]:
# Create heatmap trace
heatmap = go.Heatmap(
    x=unique_teams_list,
    y=unique_teams_list,
    z=matrix,
    colorscale='Viridis',
    colorbar=dict(title='Win Count')
)

# Create layout
layout = go.Layout(
    title='Head-to-Head Win Count',
    xaxis=dict(title='Team'),
    yaxis=dict(title='Team')
)

# Create figure
fig = go.Figure(data=[heatmap], layout=layout)

# Show figure
fig.show()